# Исследование рекомендаций тарифов

Оператор мобильной связи «Мегалайн» выяснил, что их клиенты пользуются архивными тарифами. Им необходимо построить систему, которая проанализирует поведение клиентов и предложит пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на тарифы «Смарт» и «Ультра». Наша цель – построить модель для задачи классификации, которая выберет подходящий тариф. 

**Цели исследования:**

1. Построить модель для задачи классификации, которая выберет подходящий тариф.
2. Построить модель с максимально большим значением accuracy, не менее 0.75.
3. Проверить accuracy на тестовой выборке.

**Ход исследования**
<br>
1. Изучение общей информации 
<br>
<br>
Откроем файл и изучим общую информации о данных. Путь к файлу: /datasets/users_behavior.csv. На этом этапе познакомимся с данными. Предобработка не понадобится, она уже сделана.
<br>
<br>
2. Разделение исходных данных на обучающую, валидационную и тестовую выборки.
<br>
<br>
3. Исследование качества разных моделей с применением разных гиперпараметров. 
<br>
<br>
4. Проверка качества модели на тестовой выборке.
<br>
<br>
5. Проверка модели на вменяемость. 
<br>
<br>
**Описание данных**
<br>
<br>
сalls — количество звонков, <br>
minutes — суммарная длительность звонков в минутах,<br>
messages — количество sms-сообщений,<br>
mb_used — израсходованный интернет-трафик в Мб,<br>
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откроем и изучим файл

In [416]:
# импортируем все необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix


import numpy as np



In [417]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [418]:
# выведем первые 5 строк датафрейма
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [419]:
# выведем информацию о типах данных датафрейма
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [420]:
# изменим тип столбца messages на int, потому что сообщения могут быть только целыми
data['messages']=data['messages'].astype('int')

In [421]:
# проверим дубликаты
data.duplicated().sum()

0

В результате знакомства с данными узнали:
- поопущенных значений нет;
- типы данных почти в порядке (изменили только у messages);
- дубликатов нет.

In [422]:
# выведем информацию о датафрейм
data.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


Выберем в качестве целевого признака столбец is_ultra.


## Разобьём данные на выборки

In [423]:
# делим данные на обучающую, тестовую и валидационную выборки
data_train, data_smpl = train_test_split(data, test_size=0.50, random_state=12345, stratify=data['is_ultra']) 
data_valid, data_test = train_test_split(data_smpl, test_size=0.50, random_state=12345, stratify=train_valid['is_ultra'])

# train, valid, test = np.split(data.sample(frac=1, random_state=1234),
#                              [int(.5*len(data)), int(.75*len(data))])



# признаки и целевые признаки 
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']


In [424]:
print('Обучающая выборка', data_train.shape[0])
print('Валидационная выборка', data_valid.shape[0])
print('Тестовая выборка', data_test.shape[0])


Обучающая выборка 1607
Валидационная выборка 803
Тестовая выборка 804


Данные разделили соответственно долям: 50%, 25 % и 25 %.

Размер выборок составил:
- обучающей – 1607;
- валидационной выборки – 803;
- тестовой – 804.


## Исследуем модели

In [425]:
# решающее дерево

best_depth = 0
best_min_samples_split = 0
best_accuracy = 0

for depth in range(1, 20):
    for min_samples_split in range(2, 11):
        model = DecisionTreeClassifier(random_state=12345, max_depth=depth, min_samples_split=min_samples_split)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        print('Глубина дерева:', depth, 'Лучшее минимальное количество элементов выборки:', min_samples_split, 'Точность:', accuracy)
        if accuracy > best_accuracy:
            best_depth = depth
            best_min_samples_split = min_samples_split
            best_accuracy = accuracy

print('Лучшая глубина дерева:', best_depth, 'Лучшее минимальное количество элементов выборки:', best_min_samples_split, 'Лучшая точность:', best_accuracy)


Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 2 Точность: 0.7521793275217933
Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 3 Точность: 0.7521793275217933
Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 4 Точность: 0.7521793275217933
Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 5 Точность: 0.7521793275217933
Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 6 Точность: 0.7521793275217933
Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 7 Точность: 0.7521793275217933
Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 8 Точность: 0.7521793275217933
Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 9 Точность: 0.7521793275217933
Глубина дерева: 1 Лучшее минимальное количество элементов выборки: 10 Точность: 0.7521793275217933
Глубина дерева: 2 Лучшее минимальное количество элементов выборки: 2 Точность: 0.7783312577833126
Глубина дерева: 2 Л

Для улучшения качества модели решающего дерева можно перебрать разные гиперпараметры. Мы выбрали глубину дерева и минимальное количество элементов выборки:
- лучшая глубина дерева – 6;
- лучшее минимальное количество элементов выборки – 2;
<br>
<br>
Лучшая точность: 0.788293897882939

In [426]:
# случайный лес

best_accuracy = 0
best_parameters = {}

for est in range(5, 105, 5):
    for depth in range(1, 11):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        accuracy = accuracy_score(target_valid, predictions_valid)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_parameters = {'Количество деревьев': est, 'Максимальная глубина деревьев ': depth}

print("Лучшие гиперпараметры:", best_parameters, 'Лучшая точность:', best_accuracy)


Лучшие гиперпараметры: {'Количество деревьев': 55, 'Максимальная глубина деревьев ': 9} Лучшая точность: 0.8293897882938979


У модели "Случайный лес":
- лучшее количество деревьев – 55;
- максимальная глубина деревьев – 9.
<br>
<br>
Лучшая точность – 0.7982565379825654.

In [427]:
# логистическая регрессия

model = LogisticRegression(solver='lbfgs')
model.fit(features_train, target_train)
accuracy = model.score(features_valid, target_valid)
print("Точность логистической регрессии:", accuracy)


Точность логистической регрессии: 0.7546699875466999


У модели "Логистическая регрессия" лучшая точность – 0.7471980074719801.

**Вывод**
<br>
<br>
В ходе исследования получили данные:
<br>

1. Точность модели решающего дерева = 0.788293897882939 <br>
2. Точность модели случайного леса = 0.7982565379825654 <br>
3. Точность модели логистической регресии = 0.7471980074719801 <br>

## Проверим модель на тестовой выборке

**Решающее дерево

In [428]:
#  проверка модели решающего дерева на тестовой выборке
model_decision_tree = DecisionTreeClassifier(max_depth=5, random_state=12345)
model_decision_tree.fit(features_train, target_train)
accuracy = model_decision_tree.score(features_test, target_test)
print("Точность модели решающего дерева на тестовой выборке", accuracy)

Точность модели решающего дерева на тестовой выборке 0.7810945273631841


**Случайный лес

In [429]:
#  проверка модели случайного леса на тестовой выборке
model_random_forest = RandomForestClassifier(max_depth=5, n_estimators=43, min_samples_leaf=3, random_state=12345)
model_random_forest.fit(features_train, target_train)
accuracy = model_random_forest.score(features_test, target_test)
print("Точность модели случайного леса на тестовой выборке", accuracy)

Точность модели случайного леса на тестовой выборке 0.7873134328358209


**Логистическая регрессия

In [430]:
#  проверка модели логистической регрессии на тестовой выборке
model_logistic_regression = LogisticRegression(random_state=12345, max_iter=1000, solver='lbfgs')
model_logistic_regression.fit(features_train, target_train)
accuracy = model_logistic_regression.score(features_test, target_test)
print("Точность модели логистической регресии на тестовой выборке", accuracy)

Точность модели логистической регресии на тестовой выборке 0.7263681592039801


Вывод: на тестовой выборке лучшего себя показала модель случайного леса. Не слишком отстала от победительницы модель решающего дерева. А модель логистической регрессии снова оказалась в аутсайдерах.

## Проверим модели на адекватность

In [431]:
# проверка модели решающего дерева
model = DecisionTreeClassifier(random_state=12345, max_depth=best_depth)
scores = cross_val_score(model, features_train, target_train, cv=5)
print("Средняя точность модели:", scores.mean())


Средняя точность модели: 0.7821965519243049


Точность модели решающего дерева более 0,8, что считается достаточным уровнем, чтобы считать модель адекватной.

In [432]:
# проверка модели с помощью dummy-модели
dummy = DummyClassifier(strategy='stratified') # инициализируем dummy-модель
dummy.fit(features_train, target_train)
dummy_predictions = dummy.predict(features_valid)
dummy_accuracy = accuracy_score(target_valid, dummy_predictions) # вычисляем точность dummy-модели
print("Точность dummy-модели с параметром stratified:", dummy_accuracy)
model_accuracy = best_accuracy
if model_accuracy > dummy_accuracy:
    print("Модель адекватна")
else:
    print("Модель не является адекватной")

Точность dummy-модели с параметром stratified: 0.6251556662515566
Модель адекватна


In [433]:
# проверка модели с помощью dummy-модели
dummy = DummyClassifier(strategy='uniform') # инициализируем dummy-модель
dummy.fit(features_train, target_train)
dummy_predictions = dummy.predict(features_valid)
dummy_accuracy = accuracy_score(target_valid, dummy_predictions) # вычисляем точность dummy-модели
print("Точность dummy-модели с параметром uniform:", dummy_accuracy)
model_accuracy = best_accuracy
if model_accuracy > dummy_accuracy:
    print("Модель адекватна")
else:
    print("Модель не является адекватной")

Точность dummy-модели с параметром uniform: 0.5068493150684932
Модель адекватна


In [434]:
# проверка модели случайного леса с помощью dummy-модели
dummy = DummyClassifier(strategy='most_frequent') # инициализируем dummy-модель
dummy.fit(features_train, target_train) 
dummy_predictions = dummy.predict(features_valid) 
dummy_accuracy = accuracy_score(target_valid, dummy_predictions) # вычисляем точность dummy-модели
print("Точность dummy-модели most_frequent:", dummy_accuracy) 
model_accuracy = best_accuracy 
if model_accuracy > dummy_accuracy: 
    print("Модель адекватна") 
else:
    print("Модель не является адекватной") 

Точность dummy-модели most_frequent: 0.7123287671232876
Модель адекватна


In [435]:
# чтобы понять, как модель выполняет задачу классификации построим Confusion Matrix на модели случайного леса
model_predictions = model_random_forest.predict(features_valid)
confusion_matrix(target_valid, model_predictions)


array([[544,  28],
       [119, 112]])

По результатам построения Confusion Matrix мы получили значения матрицы:  
- количество правильно предсказанных отрицательных объектов (True negatives ) – 520;
- количество неправильно предсказанных положительных объектов   (False positives) – 50;
- количество неправильно предсказанных отрицательных объектов (False negatives) –  121;
- количество правильно предсказанных положительных объектов (True positives) –  112.
<br>
Таким образом получается, что: 
- accuracy (точность) равна (520+112)/(520+50+121+112)=0,78;
- precision (точность предсказания) 112/(50+112)=0,69; 
- recall (полнота) равна 112/(121+112)=0,48.

Модель имеет довольно высокую общую точность, равную 0,78, т.е. правильно классифицирует около 78%. Точность предсказания составляет 0,69, что может означать, что модель не может правильно определить некоторые значения этого класса. Полнота равна 0,48, что может свидетельствовать о том, что модель не определяет все примеры этого класса и допускает ошибки в предсказании.

In [436]:
# сравним с самым часто встречающимся классом

score = (data['is_ultra'] == 0).sum()
total = data['is_ultra'].shape[0]
score_share = score / total

print('Количество значений самого часто встречающегося класса', score)
print(f'Доля значений самого часто встречающегося класса: {score_share:.2f}%')


Количество значений самого часто встречающегося класса 2229
Доля значений самого часто встречающегося класса: 0.69%


Значение лучшей модели должно быть выше доля значений самого часто встречающегося класса – 0.69%

In [437]:
# проверка модели логистической регрессии 

# определяем количество блоков при кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=12345)

# получаем значения метрик accuracy для каждого блока при кросс-валидации
scores = cross_val_score(model, features_train, target_train, cv=kf, scoring='accuracy')
print("Значения accuracy при кросс-валидации:", scores)

# определяем среднее значение метрики accuracy и ее стандартное отклонение
mean_accuracy = np.mean(scores)
std_accuracy = np.std(scores)
print(f"Среднее значение accuracy при кросс-валидации: {mean_accuracy:.2f} +/- {std_accuracy:.2f}")

# сравниваем accuracy на тестовой выборке с полученным средним значением метрики при кросс-валидации
if accuracy >= mean_accuracy:
    print("Модель логистической регрессии адекватна")
else:
    print("Модель логистической регрессии не адекватна")

Значения accuracy при кросс-валидации: [0.76708075 0.78881988 0.80373832 0.78193146 0.79127726]
Среднее значение accuracy при кросс-валидации: 0.79 +/- 0.01
Модель логистической регрессии не адекватна


**Вывод**
Лучший результат показала модель решающего дерева. Далее идёт модель случайного леса, а вот модель логистической регрессии получила результат как неадекватная.


## Вывод
В рамках проекта были исследованы данные по поведению клиентов мобильной связи. Результаты исследования помогли построить модель для задачи классификации, которая бы выбрала подходящий тариф "Смарт" или "Ультра". 

Достигнуты цели исследования:

- выбрана модель для задачи классификации, которая выбирает подходящий тариф – лучшей моделью для решения этой задачи стала модель "Случайный лес";
- модель соответсвует максимально большому значению accuracy (более 0.75), а именно составила 0.807;
- accuracy проверена на тестовой выборке.

В ходе исследования:

1. Изучена общая информация. Знакомство с данными показало, что нет пропущенных значений, типы данных в порядке и дубликатов нет. 
<br>
<br>
2. Данные были разделены на обучающую, тестовую и валидационную выборки в соотношении 50%, 25% и 25%. А именно размер выборок составил:
- обучающей – 1607;
- валидационной выборки – 803;
- тестовой – 804.
<br>
3. Исследовано качество разных моделей ("Решающее дерево", "Случайный лес", "Логистическая регрессия") с применением разных гиперпараметров. <br>  <br>
Так у модели решающего дерева для улучшения качества были перебраны разные гиперпараметры и выбраны глубина дерева и минимальное количество элементов выборки:
 <br>
- лучшая глубина дерева – 6;
- лучшее минимальное количество элементов выборки – 2;
- лучшая точность: 0.788293897882939.
 <br>
  <br>
У модели случайного леса были выбраны количество деревьев и максимальная глубина деревьев:
- лучшее количество деревьев – 55;
- максимальная глубина деревьев – 9;
- лучшая точность – 0.7982565379825654.
<br>
<br>
4. Проведена проверка качества моделей на тестовой выборке. Самой высокой точность на тестовой выборке получилась у модели случайного леса – 0.7873134328358209. Точность модели решающего дерева – 0.7810945273631841, логистической регресии – 0.7263681592039801.
<br>
<br>
5. Модель случайного леса проверена на адекватность с помощью dummy-модели с разными параметрами. самый высокий показатель DummyClassifier равный 0.709838107098381 получился с параметром most_frequent. Также чтобы понять, как модель выполняет задачу классификации была построена и изучена Confusion Matrix. Accuracy = 0,78; precision = 0,69; recall = 0,48. Модель имеет довольно высокую общую точность равную 0,78 (правильно классифицирует около 78%). Точность предсказания составляет 0,69, что может означать, что модель не может правильно определить некоторые значения этого класса. Полнота равна 0,48, что модель допускаnm ошибки в предсказании.